In [ ]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import uic


#랜덤맵 생성기
import random
randomInt = int(random.randrange(1,4))*5
random_map = [[0] * randomInt for _ in range(randomInt)]
for i in range(randomInt):
    for j in range(randomInt):
        random_map[i][j] = int(random.randrange(0,2))
    
#맵딕셔너리    
maps_dict = {'flower':[
                  [0,0,1,0,0],
                  [0,1,1,1,0],
                  [0,0,1,0,0],
                  [1,0,1,0,1],
                  [0,1,1,1,0]], 
             'spider':[
                     [1,0,1,0,0,0,0,1,0,1],
                     [1,0,1,0,1,1,0,1,0,1],
                     [1,0,1,0,1,1,0,1,0,1],
                     [1,0,0,1,1,1,1,0,0,1],
                     [0,1,0,1,1,1,1,0,1,0],
                     [0,0,1,1,1,1,1,1,0,0],
                     [0,1,0,1,1,1,1,0,1,0],
                     [1,0,0,0,1,1,0,0,0,1],
                     [1,0,0,1,0,0,1,0,0,1],
                     [1,0,0,1,0,0,1,0,0,1]],
             'random_map':random_map
            }

selected_num = -1   #game화면과 edit화면에서 테이블 아이템 클릭 시 반환할 맵넘버
        

###################################################################        
##라벨 클릭가능하게 만들어주는 함수. play화면과 mapedit화면에서 사용
LorR = 'N'  #좌클릭 우클릭 구분변수
def m_clicked(QLabel):
    if LorR =='L':
        if QLabel.text()=='':
            QLabel.setText('◼︎') 
        else: QLabel.setText('')
    elif LorR =='R':
        if QLabel.text()=='':
            QLabel.setText('X') 
        else: QLabel.setText('') 
    else: 
        pass    
        
        
def clickable(widget):
    global LorR
    class Filter(QObject):

        clicked = pyqtSignal()
        

        def eventFilter(self, obj, event):
            global LorR
            if obj == widget:
                if event.type() == QEvent.MouseButtonPress:
                    if event.button() == Qt.RightButton:
                        if obj.rect().contains(event.pos()):
                            LorR = 'R'
                            m_clicked(obj)
                            self.clicked.emit()
                            return True
                    elif event.button() == Qt.LeftButton:
                        if obj.rect().contains(event.pos()):
                            LorR = 'L'
                            m_clicked(obj)
                            self.clicked.emit()
                            return True
            return False
    filter = Filter(widget)
    widget.installEventFilter(filter)
    return filter.clicked

#좌클릭, 우클릭 시 표시하는 방법

        
        
###################################################################        
#시작화면 클래스
main_window_class = uic.loadUiType("Nono_main_window.ui")[0]
class main_window(QMainWindow, main_window_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        
    def MainToGame(self):     #게임하기 버튼 -> game화면
        self.hide()                    
        self.gameWindow = game_window() 
        self.gameWindow.show()
        
    def MainToEdit(self):     #편집하기 버튼 -> edit화면
        self.hide()                    
        self.editWindow = edit_window() 
        self.editWindow.show()
    
    def MainClose(self):     #종료하기 버튼 -> 프로그램 종료
        sys.exit()

###################################################################        
#게임하기화면 클래스
game_window_class = uic.loadUiType("Nono_game_window.ui")[0]
class game_window(QMainWindow,game_window_class):
    def __init__(self):
        super(game_window,self).__init__()
        self.setupUi(self)
        self.table_set()
        self.play_btn_disabled()
        self.selected_item_num()
        self.show()
    
    def table_set(self):                #maps_dict의 key와 사이즈를 Qtable에 나열
        self.tableWidget.setColumnWidth(0,350)
        self.tableWidget.setRowCount(len(maps_dict))
        for i in range(len(maps_dict)):
            self.tableWidget.setItem(i,0,QTableWidgetItem(list(maps_dict.keys())[i]))
        for j in range(len(maps_dict)):
            size = str(len(list(maps_dict.values())[j]))+'x'+str(len(list(maps_dict.values())[j]))
            self.tableWidget.setItem(j,1,QTableWidgetItem(size))
        
    def selected_item_num(self):        #맵 item 클릭 시 selected_num 변수를 행값으로 변환
        for i in self.tableWidget.selectedItems():
            global selected_num
            selected_num = i.row()
        self.play_btn_disabled()
        
    def play_btn_disabled(self):        #맵 item이 클릭됐을 때 플레이버튼 활성화
        if selected_num == -1:
            self.Play_btn.setDisabled(True)
        else: 
            self.Play_btn.setEnabled(True)     
            
    def GameToPlay(self):       #PLAY! 버튼 -> play화면
        self.hide()                    
        self.playWindow = play_window() 
        self.playWindow.show()
        
    def GameToMain(self):       #뒤로가기 버튼 -> main화면
        global selected_num
        selected_num = -1
        self.hide()                    
        self.mainWindow = main_window() 
        self.mainWindow.show()
        

###################################################################        
#Play화면 클래스
play_window_class = uic.loadUiType("Nono_play_window.ui")[0]
class play_window(QMainWindow, play_window_class):
    def __init__(self):
        super(play_window,self).__init__()
        self.setupUi(self)
        self.map_num = selected_num
        self.size = len(list(maps_dict.values())[self.map_num])
        self.pos_x = int((600-24*self.size)/2)
        self.pos_y = int((600-24*self.size)/2)
        self.squares=[]
        
        self.map_title()
        self.square()
        self.row_col_square()
    
    #제목 라벨 출력
    def map_title(self):
        title = list(maps_dict.keys())[self.map_num]
        title_label = QLabel(title, self)
        title_label.setStyleSheet('font-size:30pt;')
        title_label.resize(200,50)
        title_label.move(200,550)
        title_label.setAlignment(Qt.AlignCenter)
    
    # N*N크기의 매트릭스 생성, 각각의 라벨은 clickable하며, 클릭 이벤트 발생 시 scan_square 함수 실행.   
    def square(self):
        size = self.size
        lbl_pos_y = self.pos_y
        for i in range(size):
            temp_list=[]
            lbl_pos_x = self.pos_x
            for j in range(size):
                temp_label = QLabel('',self)
                temp_label.setStyleSheet('font-size:24pt; border: 1px solid black;')
                temp_label.resize(24,24)
                temp_label.move(lbl_pos_x,lbl_pos_y)
                temp_label.setAlignment(Qt.AlignCenter)
                temp_list.append(temp_label)
                lbl_pos_x = lbl_pos_x+24
            lbl_pos_y = lbl_pos_y+24
            self.squares.append(temp_list)
        for i in range(len(self.squares)):
            for j in self.squares[i]:
                clickable(j).connect(self.scan_square)

    # 라벨 클릭이벤트 발생시마다 변경된 맵을 스캔하여 map_dict의 초기정보와 일치하는지 확인, 일치하면 clear 메시지 팝업.
    def scan_square(self):
        scanned_square=[]
        for i in range(len(self.squares)):
            scanned_list=[]
            for j in self.squares[i]:
                if j.text()=='◼︎':
                    scanned_list.append(1)
                else: scanned_list.append(0)
            scanned_square.append(scanned_list)
        if scanned_square == list(maps_dict.values())[selected_num]:
            QMessageBox.about(self,'message','clear!')

    # 3. 플레이맵의 좌측과 상단에 표시될 행과 열의 연속된 색칠칸 정보를 표현하는 함수 작성, ex. [0,1,0,1,1] ->[1,2]
    def row_col_square(self):
        row_list = self.row_num()
        lbl_pos_x = self.pos_x-24*(len(row_list)-1)
        lbl_pos_y = self.pos_y
        
        for i in range(self.size): 
            num_list = row_list[i]
            nums = ' '.join(map(str, num_list))
            temp_label = QLabel(nums,self)
            temp_label.resize(24*(len(row_list)-1),24)
            temp_label.setAlignment(Qt.AlignRight)
            temp_label.move(lbl_pos_x-5,lbl_pos_y+3)
            lbl_pos_y += 24     
        col_list = self.col_num()
        lbl_pos_x = self.pos_x
        lbl_pos_y = self.pos_y-24*(len(row_list)-1)
        
        for i in range(self.size):
            num_list = col_list[i]
            nums = '\n'.join(map(str, num_list))
            temp_label = QLabel(nums,self)
            temp_label.resize(24,24*(len(col_list)-1))
            temp_label.setAlignment(Qt.AlignCenter)
            temp_label.setAlignment(Qt.AlignBottom)
            temp_label.move(lbl_pos_x+9,lbl_pos_y-2)
            lbl_pos_x += 24           
            
    def row_num(self):
        li = list(maps_dict.values())[self.map_num]
        row_nums = []
        
        for i in range(len(li)):
            li_temp = li[i]+[0]
            li_ = []
            nTemp = 0
            for j in range(len(li_temp)):
                if li_temp[j] == 1:
                    nTemp +=1
                if li_temp[j] == 0:
                    if nTemp!=0 : li_.append(nTemp)
                    nTemp=0
            row_nums.append(li_)
        return row_nums
    
    def col_num(self):
        col_nums = []
        li = list(maps_dict.values())[self.map_num]
        li = list(map(list,list(zip(*li))))
        for i in range(len(li)):
            li_temp = li[i]+[0]
            li_ = []
            nTemp = 0
            for j in range(len(li_temp)):
                if li_temp[j] == 1:
                    nTemp +=1
                if li_temp[j] == 0:
                    if nTemp!=0 : li_.append(nTemp)
                    nTemp=0
            col_nums.append(li_)
        return col_nums
    
    
    #뒤로가기 버튼 -> game화면
    def PlayToGame(self):
        global selected_num
        selected_num = -1
        self.hide()
        self.gameWindow = game_window()
        self.gameWindow.show()

        

###################################################################       
#편집화면 클래스
edit_window_class = uic.loadUiType("Nono_edit_window.ui")[0]
class edit_window(QMainWindow,edit_window_class):
    def __init__(self):
        super(edit_window,self).__init__()
        global selected_num
        selected_num = -1
        self.setupUi(self)
        self.table_set()
        self.Modi_btn_disabled()
        self.selected_item_num()
        self.show()
        
     #maps_dict의 key와 사이즈를 Qtable에 나열
    def table_set(self):               
        self.tableWidget.setColumnWidth(0,350)
        self.tableWidget.setRowCount(len(maps_dict))
        for i in range(len(maps_dict)):
            self.tableWidget.setItem(i,0,QTableWidgetItem(list(maps_dict.keys())[i]))
        for j in range(len(maps_dict)):
            size = str(len(list(maps_dict.values())[j]))+'x'+str(len(list(maps_dict.values())[j]))
            self.tableWidget.setItem(j,1,QTableWidgetItem(size))
    
    #맵 item 클릭 시 selected_num 변수를 행값으로 변환    
    def selected_item_num(self):        
        for i in self.tableWidget.selectedItems():
            global selected_num
            selected_num = i.row()
        self.Modi_btn_disabled()
    
    #맵 item이 클릭됐을 때 플레이버튼 활성화
    def Modi_btn_disabled(self):        
        if selected_num == -1:
            self.Modi_btn.setDisabled(True)
            self.Del_btn.setDisabled(True)
        else: 
            self.Modi_btn.setEnabled(True)
            self.Del_btn.setEnabled(True)     
    
    #추가하기 버튼 -> mapedit화면
    def EditToMapEdit_add(self):
        self.hide()
        self.mapeditAddWindow = mapedit_add_window()
        self.mapeditAddWindow.show()
    
    #수정하기 버튼 -> mapedit화면
    def EditToMapEdit(self):    
        self.hide()
        self.mapeditWindow = mapedit_window()
        self.mapeditWindow.show()


            
    #뒤로가기 버튼 -> main화면
    def EditToMain(self):         
        self.hide()                    
        self.mainWindow = main_window() 
        self.mainWindow.show()
        
    #삭제하기 버튼 -> 선택된 맵 삭제    
    def MapDelete(self):
        global selected_num
        del_key = list(maps_dict.keys())[selected_num]
        del maps_dict[del_key]
        selected_num = -1
        self.table_set()
        self.Modi_btn_disabled()
    
###################################################################
#맵편집화면 클래스(추가하기)
mapedit_window_class = uic.loadUiType("Nono_mapedit_add_window.ui")[0]
class mapedit_add_window(QMainWindow, mapedit_window_class):
    def __init__(self):
        super(mapedit_add_window,self).__init__()
        self.map_num = len(maps_dict)+1
        self.setupUi(self)
        self.squares=[]
        #####
        self.row_label_list=[]
        self.col_label_list=[]
        #####
        self.new_map = []

        self.text = str(self.comboBox.currentText())
        if self.text =='5x5':
            self.size = 5
        elif self.text =='10x10':
            self.size = 10
        elif self.text =='15x15':
            self.size = 15
        else: print('wrong size')
        for i in range(self.size):
            temp_new_list=[]
            for j in range(self.size):
                temp_new_list.append(0)
            self.new_map.append(temp_new_list)    
        self.pos_x = int((600-24*self.size)/2)
        self.pos_y = int((600-24*self.size)/2)
        
        self.square()
        self.show()
    
    #맵 제목
    def set_map_title(self):
        title = self.lineEdit.text()
        return title
        
    #N*N 매트릭스 생성
    def square(self):
        self.squares=[]
        size = self.size
        self.pos_x = int((600-24*self.size)/2)
        self.pos_y = int((600-24*self.size)/2)
        lbl_pos_y = self.pos_y
        for i in range(size):
            temp_list=[]
            lbl_pos_x = self.pos_x
            for j in range(size):
                temp_label = QLabel('',self)
                temp_label.setStyleSheet('font-size:24pt; border: 1px solid black;')
                temp_label.resize(24,24)
                temp_label.move(lbl_pos_x,lbl_pos_y)
                temp_label.setAlignment(Qt.AlignCenter)
                temp_list.append(temp_label)
                temp_label.setVisible(True)
                lbl_pos_x = lbl_pos_x+24
            lbl_pos_y = lbl_pos_y+24
            self.squares.append(temp_list)
        for i in range(len(self.squares)):
            for j in self.squares[i]:
                clickable(j).connect(self.edit_square)
    
    #칸 클릭시마다 변경된 매트릭스정보를 저장
    def edit_square(self):
        edited_square=[]
        for i in range(len(self.squares)):
            edited_list=[]
            for j in self.squares[i]:
                if j.text()=='◼︎':
                    edited_list.append(1)
                else: edited_list.append(0)
            edited_square.append(edited_list)
        self.new_map = edited_square
        self.row_col_square()           
        
        #변경된 매트릭스에 따라 행과 열의 숫자정보 표시
    def row_col_square(self):
        for i in range(len(self.row_label_list)):
            self.row_label_list[i].hide()
            self.col_label_list[i].hide()
        #row
        row_list = self.row_num()
        lbl_pos_x = self.pos_x-24*(len(row_list)-1)
        lbl_pos_y = self.pos_y
        for i in range(self.size): 
            num_list = row_list[i]
            nums = ' '.join(map(str, num_list))
            temp_label = QLabel(nums,self)
            temp_label.resize(24*(len(row_list)-1),24)
            temp_label.setAlignment(Qt.AlignRight)
            temp_label.move(lbl_pos_x-5,lbl_pos_y+3)
            temp_label.setVisible(True)
            self.row_label_list.append(temp_label)
            lbl_pos_y += 24
        
        #col    
        col_list = self.col_num()
        lbl_pos_x = self.pos_x
        lbl_pos_y = self.pos_y-24*(len(row_list)-1)
        for i in range(self.size):
            num_list = col_list[i]
            nums = '\n'.join(map(str, num_list))
            temp_label = QLabel(nums,self)
            temp_label.resize(24,24*(len(col_list)-1))
            temp_label.setAlignment(Qt.AlignCenter)
            temp_label.setAlignment(Qt.AlignBottom)
            temp_label.move(lbl_pos_x+9,lbl_pos_y-2)
            temp_label.setVisible(True)
            self.col_label_list.append(temp_label)
            lbl_pos_x += 24           
            
    def row_num(self):
        li = self.new_map
        row_nums = []
        
        for i in range(len(li)):
            li_temp = li[i]+[0]
            li_ = []
            nTemp = 0
            for j in range(len(li_temp)):
                if li_temp[j] == 1:
                    nTemp +=1
                if li_temp[j] == 0:
                    if nTemp!=0 : li_.append(nTemp)
                    nTemp=0
            row_nums.append(li_)
        return row_nums
    
    def col_num(self):
        col_nums = []
        li = self.new_map
        li = list(map(list,list(zip(*li))))
        for i in range(len(li)):
            li_temp = li[i]+[0]
            li_ = []
            nTemp = 0
            for j in range(len(li_temp)):
                if li_temp[j] == 1:
                    nTemp +=1
                if li_temp[j] == 0:
                    if nTemp!=0 : li_.append(nTemp)
                    nTemp=0
            col_nums.append(li_)
        return col_nums

    def remap_for_size(self):

        for i in range(len(self.squares)):
            for j in self.squares[i]:
                j.hide()
        for i in range(len(self.row_label_list)):
            self.row_label_list[i].hide()
            self.col_label_list[i].hide()
            
        self.text = str(self.comboBox.currentText())
        if self.text =='5x5':
            self.size = 5
        elif self.text =='10x10':
            self.size = 10
        elif self.text =='15x15':
            self.size = 15
        else:
            print('wrong size')
        

        self.square()        

    
    #뒤로가기 버튼 -> edit화면
    def MapEditToEdit(self):     
        self.hide()
        self.editWindow = edit_window()
        self.editWindow.show()
    
    #작업완료 후 edit화면으로 자동 복귀
    def MapSave(self):
        map_title = self.set_map_title()
        maps_dict[map_title]=self.new_map

        self.MapEditToEdit()    
        
    #작업완료 후 edit화면으로 자동 복귀
    def MapDelete(self):
        global selected_num
        selected_num = -1
        self.MapEditToEdit()         

###################################################################
#맵편집화면 클래스(수정하기)
mapedit_window_class = uic.loadUiType("Nono_mapedit_window.ui")[0]
class mapedit_window(QMainWindow, mapedit_window_class):
    def __init__(self):
        super(mapedit_window,self).__init__()
        self.setupUi(self)
        self.squares=[]
        self.square()
        default_title=list(maps_dict.keys())[selected_num]
        self.lineEdit.setText(default_title)
        self.show()
    
    #맵 제목
    def set_map_title(self):
        title = self.lineEdit.text()
        return title
    
    def square(self):
        maps_info = list(maps_dict.values())[selected_num]
        size = len(list(maps_dict.values())[selected_num])
        pos_x = int((600-24*size)/2)
        pos_y = int((600-24*size)/2)
        lbl_pos_y = pos_y
        for i in maps_info:
            temp_list=[]
            lbl_pos_x = pos_x
            for j in i:
                if j==1: temp_label = QLabel('◼︎',self)
                else:    temp_label = QLabel('',self)
                temp_label.setStyleSheet('font-size:24pt; border: 1px solid black;')
                temp_label.resize(24,24)
                temp_label.move(lbl_pos_x,lbl_pos_y)
                temp_label.setAlignment(Qt.AlignCenter)
                temp_label.setVisible(True)
                temp_list.append(temp_label)
                lbl_pos_x = lbl_pos_x+24
            lbl_pos_y = lbl_pos_y+24
            self.squares.append(temp_list)
        for i in range(len(self.squares)):
            for j in self.squares[i]:
                clickable(j).connect(self.save_square)
                
                
    def save_square(self):
        saved_square=[]
        for i in range(len(self.squares)):
            saved_list=[]
            for j in self.squares[i]:
                if j.text()=='◼︎':
                    saved_list.append(1)
                else: saved_list.append(0)
            saved_square.append(saved_list)
        return saved_square
            
    #뒤로가기 버튼 -> edit화면
    
    def MapEditToEdit(self):     
        self.hide()
        self.editWindow = edit_window()
        self.editWindow.show()
    
    #작업완료 후 edit화면으로 자동 복귀
    def MapSave(self):
        global maps_dict
        global selected_num
        present_key=list(maps_dict.keys())[selected_num]
        del maps_dict[present_key]
        saved_map = self.save_square()
        edited_map_title = self.set_map_title()
        maps_dict[edited_map_title] = saved_map
        self.MapEditToEdit()    
        selected_num = -1
        
    #작업완료 후 edit화면으로 자동 복귀
    def MapDelete(self):
        global selected_num
        del_key = list(maps_dict.keys())[selected_num]
        del maps_dict[del_key]
        selected_num = -1
        self.MapEditToEdit()     

###################################################################        
#프로그램 실행
if __name__ == '__main__':
    app = QApplication(sys.argv)
    mainWindow = main_window()
    mainWindow.show()
    app.exec_()
    
    


2022-07-22 10:15:43.738 python[49654:1861556] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
